In [ ]:
import os

import pandas as pd
import numpy as np

dir_path = os.path.abspath('')
data_path = os.path.abspath(os.path.join(dir_path, '..', "data"))
results_path = os.path.abspath(os.path.join(dir_path, '..', "results"))
figures_path = os.path.abspath(os.path.join(dir_path, '..', "figures"))
tables_path = os.path.abspath(os.path.join(dir_path, '..', "tables"))
import ast

import re
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc

uav_table_order = [ 'uav-8', 'uav-8-OC', 'uav-9', 'uav-9-OC', 'uav-10', 'uav-10-OC']

# rc('font', **{'family': 'serif', 'serif': ['Computer Modern'], 'size':16})
font = {'family' : 'normal',
        'size'   : 22}

rc('font', **font)
rc('text', usetex=True)

# Clean and compile file

In [ ]:
def get_files_by_problem_types() -> pd.DataFrame:
    all_results = os.listdir(results_path)
    full_results = []
    for file_name in all_results:
        if file_name[-4:] != ".pkl":
            continue
        rounds_match = re.match(rf".*_([\d|None]+).pkl", file_name)
        assert rounds_match is not None
        df = pd.read_pickle(f"{results_path}/{file_name}")
        full_results.append(df)

    full_results_df = pd.concat(full_results)

    return full_results_df

In [ ]:
df = get_files_by_problem_types()
# Make column of 
df['problem_arguments'] = df['problem_arguments'].apply(lambda x: ast.literal_eval(x))
df = pd.concat([df.drop(['problem_arguments'], axis=1), df['problem_arguments'].apply(pd.Series)], axis=1)
df['number_of_rounds'].fillna(np.inf, inplace=True)
df.loc[df['problem'] == "uav", "num_scenarios"] = df.loc[df['problem'] == 'uav', 'l']**2
assert (df.groupby(['number_of_rounds', "name"])["num_scenarios"].mean() == df.groupby(['number_of_rounds', "name"])["item_"].nunique()).all()
scenario_check = df.groupby(['number_of_rounds', "name"])["item_"].agg(["count", "nunique"])

In [ ]:
option = 1
if option == 0:
    df = df[(df['number_of_rounds'] < 50) | (df['number_of_rounds'] > 999999)]
else:
    df = df[df['number_of_rounds'] < 999999]
    df.loc[df['number_of_rounds'] > 50, 'number_of_rounds'] = np.inf

In [ ]:
fully_adaptive_cost = df.loc[np.isinf(df['number_of_rounds']), ['name', 'item_', 'grand_tour_cost']]
df = df.merge(fully_adaptive_cost, on=['name', 'item_'], suffixes=('', '_fully_adaptive'))
df['relative_difference'] = (df['grand_tour_cost'] - df['grand_tour_cost_fully_adaptive'])/df['grand_tour_cost_fully_adaptive'] * 100

In [ ]:
df["name"].replace({"uav-10-oc": "uav-10-OC"}, inplace=True)

In [ ]:
df["name"].replace({"road-60-50": "road-60-1"}, inplace=True)
df["name"].replace({"road-60-50-2": "road-60-2"}, inplace=True)
df["name"].replace({"road-62-50": "road-62-1"}, inplace=True)
df["name"].replace({"road-62-50-2": "road-62-2"}, inplace=True)
df["name"].replace({"road-64-50": "road-64-1"}, inplace=True)
df["name"].replace({"road-64-50-2": "road-64-2"}, inplace=True)
df["name"].replace({"road-66-50": "road-66-1"}, inplace=True)
df["name"].replace({"road-66-50-2": "road-66-2"}, inplace=True)
df["name"].replace({"road-68-50": "road-68-1"}, inplace=True)
df["name"].replace({"road-68-50-2": "road-68-2"}, inplace=True)

In [ ]:
df.head()

Eyeball that the number of scenario make sense

In [ ]:
scenario_check.unstack()


In [ ]:
# Check all data are collected.
print(50*12)
print(64*9)
df.groupby(["name"], as_index=False)["number_of_rounds"].count()


## Compute the planning time, with first tour amortized.

In [ ]:
k_rounds_indices = df['first_tour_planning_time'].notna()

# Sanity check
first_entry =  df.loc[k_rounds_indices, 'planning_time_record'].map(lambda x: x[0]) 
assert (first_entry.isnull()).all()


df.loc[k_rounds_indices, "first_tour_planning_time"] = df.loc[k_rounds_indices, "first_tour_planning_time"] / 10e9
df.loc[k_rounds_indices, "first_tour_planning_time_amortized"] = df.loc[k_rounds_indices, 'first_tour_planning_time'] / (df.loc[k_rounds_indices, 'num_scenarios'])

df['planning_time_record'] = df['planning_time_record'].apply(lambda x:[t/10e9 for t in x if t is not None])

df.loc[k_rounds_indices, 'planning_time_record'] = df.loc[k_rounds_indices].apply(lambda x: [x['first_tour_planning_time_amortized']] + x['planning_time_record'], axis=1)
df['Total_planning_time'] = df['planning_time_record'].apply(lambda x: sum(x))

In [ ]:
df.rename({'visited': "Visited", 
           'partial_realization': "Partial Realization",
           'item_': "Item", 
           'grand_tour_cost': "Path Cost",
       'iteration': "Iteration Taken", 
       'time_taken': "Time Taken (s)", 
       'planning_time_record': "Planning Time Per Round (s)",
       'first_tour_planning_time': "Planning Time for First Tour (s)", 
       'name': "Problem Name", 
       'problem': "Problem Type", 
       'l': "Length of Grid", 
       'number_of_rounds': "Number of Rounds",
       'file_name': "File Name", 
       'num_scenarios': "Number of Scenarios",
       'occlusions': "Occlusions",
       'grand_tour_cost_fully_adaptive': "Fully Adaptive Path Cost", 
       'relative_difference': "Relative Cost Per Scenario (vs. Fully Adaptive)",
       'first_tour_planning_time_amortized': "Amortized Planning Time for First Tour (s)", 
       'Total_planning_time': "Planning Time (s)"}, inplace=True, axis=1)

# Analysis

## Runtime

In [ ]:
#df['Number of Rounds'] = df['Number of Rounds'].replace(np.inf, 'n')


In [ ]:

road_data = df[df['Problem Type'].str.contains("road")]
uav_data = df[df['Problem Type'].str.contains("uav")]

# SMALL SIZE FOR POSTER 
# road_data = road_data[~road_data['Problem Name'].str.endswith("2") | (road_data['Problem Name'] == "road-60-50-2")]

###########

hue_order = sorted(road_data['Problem Name'].unique())

### Road Runtime Table

In [ ]:
road_time_summary = road_data.groupby(["Problem Name", "Number of Rounds"], as_index=False)["Planning Time (s)"].mean()
road_time_summary_table = road_time_summary.set_index(["Problem Name", "Number of Rounds"]).unstack()
road_time_summary_table

In [ ]:
s = road_time_summary_table.style.format(precision=2).format_index(precision=0, axis=1)

print(s.to_latex(buf=f'{tables_path}/road_time_table.tex',
                 hrules=True, 
                 multicol_align="c",
                 environment='table', 
                 caption='Average Planning Time of Road Networks Across All Scenarios', 
                 label='tab:road_time', position='htbp', 
                 position_float='centering'))

### UAV Runtime Table

In [ ]:
uav_time = uav_data.groupby(["Problem Name", "Number of Rounds"], as_index=False)["Planning Time (s)"].mean()

uav_time_summary = uav_time.set_index(["Problem Name", "Number of Rounds"]).unstack()
uav_time_summary = uav_time_summary.reindex(uav_table_order)
uav_time_summary


In [ ]:
s = uav_time_summary.style.format(precision=2).format_index(precision=0, axis=1)

s.to_latex(buf=f'{tables_path}/uav_time_table.tex',
            hrules=True, 
            multicol_align="c", 
            environment='table', 
            caption='Average Planning of UAV Time Across All Scenarios', 
            label='tab:uav-time', 
            position='htbp', 
            position_float='centering')

### Road Runtime Figure

In [ ]:
road_data

In [ ]:
fig1, ax1 = plt.subplots(figsize=(10, 6))
sns.pointplot(x='Number of Rounds', y='Planning Time (s)', hue="Problem Name", data=road_data, legend="full", ax=ax1, hue_order = hue_order,linewidth = 1.5, errorbar=None)
sns.move_legend(ax1, "lower center", bbox_to_anchor=(0.47, 1), ncol=3, title=None, frameon=False)
ax1.set_xticklabels(['{:.0f}'.format(float(t.get_text()))  if float(t.get_text()) < 100000 else 'n' for t in ax1.get_xticklabels()])
ax1.set_ylabel("Average Planning Time (s)")
fig1.savefig(f"{figures_path}/road_time.pdf", bbox_inches='tight')

### UAV Runtime Figure

In [ ]:
fig2, ax2 = plt.subplots(figsize=(10, 6))
sns.pointplot(x='Number of Rounds', y='Planning Time (s)', hue="Problem Name", data=uav_data, legend="full", ax=ax2,linewidth = 1.5, errorbar=None, hue_order=uav_table_order)
sns.move_legend(ax2, "lower center", bbox_to_anchor=(0.47, 1), ncol=3, title=None, frameon=False)
ax2.set_xticklabels(['{:.0f}'.format(float(t.get_text())) if float(t.get_text()) < 100000 else 'n' for t in ax2.get_xticklabels()])
fig2.savefig(f"{figures_path}/uav_time.pdf", bbox_inches='tight')

## Cost

### Road Cost Table

In [ ]:
road_cost_summary = road_data.groupby(["Problem Name", "Number of Rounds"], as_index=False)["Path Cost"].mean()
road_cost_summary_table = road_cost_summary.set_index(["Problem Name", "Number of Rounds"]).unstack()
road_cost_summary_table = road_cost_summary_table/1000

In [ ]:
road_cost_summary_table

In [ ]:
s = road_cost_summary_table.style.format(precision=0).format_index(precision=0, axis=1)

s.to_latex(
    buf=f'{tables_path}/road_cost_table.tex',
    hrules=True,
    multicol_align="c", 
    environment='table', 
    caption='Average Cost for Road Networks (in km)', 
    label='tab:road_cost', 
    position='htbp', 
    position_float='centering')

### UAV Cost Table

In [ ]:
uav_cost_summary = uav_data.groupby(["Problem Name", "Number of Rounds"], as_index=False)["Path Cost"].mean()
uav_cost_summary_table = uav_cost_summary.set_index(["Problem Name", "Number of Rounds"]).unstack().reindex(uav_table_order)

uav_cost_summary_table

In [ ]:
s = uav_cost_summary_table.style.format(precision=2).format_index(precision=0, axis=1)
s.to_latex(
        buf=f'{tables_path}/uav_cost_table.tex',
        hrules=True,
        multicol_align="c", 
        environment='table', 
        caption='Average Cost UAV', 
        label='tab:uav_cost', 
        position='htbp', 
        position_float='centering'
    )

### Road Cost Figure

In [ ]:
fig3, ax3 = plt.subplots(figsize=(10, 6))
hue_order = sorted(road_data['Problem Name'].unique())
sns.pointplot(x='Number of Rounds', y='Path Cost', hue="Problem Name", data=road_data, legend="full", ax=ax3, hue_order = hue_order,linewidth = 1.5, errorbar=None)
sns.move_legend(ax3, "lower center", bbox_to_anchor=(0.47, 1), ncol=5, title=None, frameon=False)
ax3.set_xticklabels(['{:.0f}'.format(float(t.get_text())) if float(t.get_text()) < 100000 else 'n' for t in ax3.get_xticklabels()])
fig3.savefig(f"{figures_path}/road_cost.pdf", bbox_inches='tight')

### UAV Cost Figure

In [ ]:
fig4, ax4 = plt.subplots(figsize=(10, 6))
sns.pointplot(x='Number of Rounds', y='Path Cost', hue="Problem Name", data=uav_data, legend="full", ax=ax4, hue_order = uav_table_order,linewidth = 1.5, errorbar=None)
sns.move_legend(ax4, "lower center", bbox_to_anchor=(0.47, 1), ncol=3, title=None, frameon=False)
ax4.set_xticklabels(['{:.0f}'.format(float(t.get_text()))  if float(t.get_text()) < 100000 else 'n' for t in ax4.get_xticklabels()])
fig4.savefig(f"{figures_path}/uav_cost.pdf", bbox_inches='tight')

## Relative Improvement Per Scenario

### Road Relative Cost Table

In [ ]:
road_rc_summary = road_data.groupby(["Problem Name", "Number of Rounds"], as_index=False)["Relative Cost Per Scenario (vs. Fully Adaptive)"].mean()
road_rc_summary_table = road_rc_summary.set_index(["Problem Name", "Number of Rounds"]).unstack()
road_rc_summary_table

In [ ]:
road_rc_summary_table.mean()

In [ ]:
s = road_rc_summary_table.style.format(precision=2).format_index(precision=0, axis=1)

s.to_latex(
    buf=f'{tables_path}/road_arc_table.tex',
    hrules=True, 
    multicol_align="c", 
    environment='table', 
    caption='Average Relative Cost Per Scenario (vs. Fully Adaptive) for Road Networks', 
    label='tab:road_arc', 
    position='htbp', 
    position_float='centering')

### UAV Relative Cost Table

In [ ]:
uav_rc_summary = uav_data.groupby(["Problem Name", "Number of Rounds"], as_index=False)["Relative Cost Per Scenario (vs. Fully Adaptive)"].mean()
uav_rc_summary_table = uav_rc_summary.set_index(["Problem Name", "Number of Rounds"]).unstack().reindex(uav_table_order)
uav_rc_summary_table

In [ ]:
uav_rc_summary_table.mean()

In [ ]:
s = uav_rc_summary_table.style.format(precision=2).format_index(precision=0, axis=1)
s.to_latex(
    buf=f'{tables_path}/uav_arc_table.tex',
    hrules=True, 
    multicol_align="c", 
    environment='table', 
    caption='Average Relative Cost Per Scenario (vs. Fully Adaptive) for UAV Search', 
    label='tab:uav_arc', 
    position='htbp',
    position_float='centering')

### Road Relative Cost Figure

In [ ]:
fig5, ax5 = plt.subplots(figsize=(10, 6))
hue_order = sorted(road_data['Problem Name'].unique())
sns.pointplot(x='Number of Rounds', y='Relative Cost Per Scenario (vs. Fully Adaptive)', hue="Problem Name", data=road_data, legend="full", ax=ax5, hue_order = hue_order,linewidth = 1.5, errorbar=None)
sns.move_legend(ax5, "lower center", bbox_to_anchor=(0.5, 1), ncol=3, title=None, frameon=False)
ax5.set_xticklabels(['{:.0f}'.format(float(t.get_text())) if float(t.get_text()) < 100000 else 'n' for t in ax5.get_xticklabels()])
ax5.set_ylabel("Average Relative Cost Per Scenario")
fig5.savefig(f"{figures_path}/road_arc.pdf", bbox_inches='tight')

### UAV Relative Cost Figure

In [ ]:
fig6, ax6 = plt.subplots(figsize=(10, 6))
sns.pointplot(x='Number of Rounds', y='Relative Cost Per Scenario (vs. Fully Adaptive)', hue="Problem Name", data=uav_data, legend="full", ax=ax6, hue_order = uav_table_order,linewidth = 1.5, errorbar=None)
sns.move_legend(ax6, "lower center", bbox_to_anchor=(0.5, 1), ncol=3, title=None, frameon=False)
ax6.set_ylabel("Average Relative Cost Per Scenario")
ax6.set_xticklabels(['{:.0f}'.format(float(t.get_text())) if float(t.get_text()) < 100000 else 'n'  for t in ax6.get_xticklabels()])
fig6.savefig(f"{figures_path}/uav_arc.pdf", bbox_inches='tight')

-Reduce the detail of uav and star

-remove star

-The ICM is parametarized by edge probabilities and we ... 

-Report the expected cost and the average computational time over all scenario. One would expect that the expected cost decreases with the number of rounds

- While there is a general trend that the ... decrease with the number of cost. Randomness based on stochastic tree embedding.

Thm 4.1. There is a randomized k-rounds algorithm for hypothesis identification. 

Our algorithm from thm 4.1 is randomized because of the probabilistic tree embedding. 

The table is misleading. 

Explain that there is a genearl trend, some of them there is a more significant reduction that others.

Explain that fully adaptive. Instances is 1 paragraph. Tested algorithms. Reported quantities: expected cost (give the equation). 


## Relative Improvement

### Road Relative Imrpovement Table

In [ ]:
road_ri_summary = road_data.groupby(["Problem Name", "Number of Rounds"], as_index=False)[["Path Cost", "Fully Adaptive Path Cost"]].mean()
road_ri_summary["Relative Average Cost (vs. Fully Adaptive)"] = (road_ri_summary["Path Cost"]- road_ri_summary["Fully Adaptive Path Cost"] )/road_ri_summary["Fully Adaptive Path Cost"] * 100
road_ri_summary.drop(["Path Cost", "Fully Adaptive Path Cost"], axis=1, inplace=True)
road_ri_summary_table = road_ri_summary.set_index(["Problem Name", "Number of Rounds"]).unstack().reindex(hue_order)
road_ri_summary_table

In [ ]:
s = road_ri_summary_table.style.format(precision=2).format_index(precision=0, axis=1)

s.to_latex(
    buf=f'{tables_path}/road_rac_table.tex',
    hrules=True, 
    multicol_align="c", 
    environment='table', 
    caption='Relative Average Cost (vs. Fully Adaptive) for Road Networks', 
    label='tab:road_rac', 
    position='htbp', 
    position_float='centering')

### UAV Relative Improvement Table

In [ ]:
uav_ri_summary = uav_data.groupby(["Problem Name", "Number of Rounds"], as_index=False)[["Path Cost", "Fully Adaptive Path Cost"]].mean()
uav_ri_summary["Relative Average Cost (vs. Fully Adaptive)"] = (uav_ri_summary["Path Cost"]- uav_ri_summary["Fully Adaptive Path Cost"] )/uav_ri_summary["Fully Adaptive Path Cost"] * 100
uav_ri_summary.drop(["Path Cost", "Fully Adaptive Path Cost"], axis=1, inplace=True)
uav_ri_summary_table = uav_ri_summary.set_index(["Problem Name", "Number of Rounds"]).unstack().reindex(uav_table_order)
uav_ri_summary_table

In [ ]:
s = uav_ri_summary_table.style.format(precision=2).format_index(precision=0, axis=1)

s.to_latex(
    buf=f'{tables_path}/uav_rac_table.tex',
    hrules=True, 
    multicol_align="c", 
    environment='table', 
    caption='Relative Average Cost (vs. Fully Adaptive) for UAV Search', 
    label='tab:uav_rac', 
    position='htbp', 
    position_float='centering')

### Road Relative Improvement Figure

In [ ]:
fig7, ax7 = plt.subplots(figsize=(10, 6))
hue_order = sorted(road_data['Problem Name'].unique())
sns.pointplot(x='Number of Rounds', y='Relative Average Cost (vs. Fully Adaptive)', hue="Problem Name", data=road_ri_summary, legend="full", ax=ax7, hue_order = hue_order,linewidth = 1.5, errorbar=None)
sns.move_legend(ax7, "lower center", bbox_to_anchor=(0.5, 1), ncol=4, title=None, frameon=False)
ax7.set_xticklabels(['{:.0f}'.format(float(t.get_text())) if float(t.get_text()) < 100000 else 'n' for t in ax7.get_xticklabels()])
fig7.savefig(f"{figures_path}/road_rac.pdf", bbox_inches='tight')

In [ ]:
fig8, ax8 = plt.subplots(figsize=(10, 6))
sns.pointplot(x='Number of Rounds', y='Relative Average Cost (vs. Fully Adaptive)', hue="Problem Name", data=uav_ri_summary, legend="full", ax=ax8, hue_order = uav_table_order,linewidth = 1.5, errorbar=None)
sns.move_legend(ax8, "lower center", bbox_to_anchor=(0.5, 1), ncol=3, title=None, frameon=False)

ax8.set_xticklabels(['{:.0f}'.format(float(t.get_text())) if float(t.get_text()) < 100000 else 'n' for t in ax8.get_xticklabels()])
fig8.savefig(f"{figures_path}/uav_rac.pdf", bbox_inches='tight')

## Road Figures Combined

In [ ]:
fig9, ax9 = plt.subplots(figsize=(10, 4))
sns.pointplot(x='Number of Rounds', y='Relative Average Cost (vs. Fully Adaptive)', hue="Problem Name", data=road_ri_summary, legend="full", ax=ax9, hue_order = hue_order,linewidth = 1.5, errorbar=None)

ax9.set_ylabel("Relative Average Cost")
ax9.set_xticklabels(['{:.0f}'.format(float(t.get_text())) for t in ax9.get_xticklabels()])
fig9.savefig(f"{figures_path}/road_combined_rac.pdf", bbox_inches='tight')

In [ ]:
fig10, ax10 = plt.subplots(figsize=(10, 4))
sns.pointplot(x='Number of Rounds', y='Relative Cost Per Scenario (vs. Fully Adaptive)', hue="Problem Name", data=road_data, ax=ax10, hue_order = hue_order,linewidth = 1.5, errorbar=None, legend=None)
ax10.set_ylabel("Average Relative Cost \nPer Scenario")
ax10.set_xticklabels(['{:.0f}'.format(float(t.get_text())) if float(t.get_text()) < 100000 else 'n' for t in ax10.get_xticklabels()])
fig10.savefig(f"{figures_path}/road_combined_arc.pdf", bbox_inches='tight')

In [ ]:
fig11, ax11 = plt.subplots(figsize=(10, 4))
sns.pointplot(x='Number of Rounds', y='Planning Time (s)', hue="Problem Name", data=road_data, ax=ax11, hue_order = hue_order,linewidth = 1.5, errorbar=None)
sns.move_legend(ax11, "lower center", bbox_to_anchor=(0.45, 1), ncol=4, title=None, frameon=False)
ax11.set_xticklabels(['{:.0f}'.format(float(t.get_text())) if float(t.get_text()) < 100000 else 'n' for t in ax11.get_xticklabels()])
fig11.savefig(f"{figures_path}/road_combined_time.pdf", bbox_inches='tight')

In [ ]:
fig12, ax12 = plt.subplots(figsize=(10, 4))
sns.pointplot(x='Number of Rounds', y='Relative Cost Per Scenario (vs. Fully Adaptive)', hue="Problem Name", data=uav_data, legend=None, ax=ax12, hue_order = uav_table_order,linewidth = 1.5, errorbar=None)
ax12.set_ylabel("Average Relative Cost\nPer Scenario")
ax12.set_xticklabels(['{:.0f}'.format(float(t.get_text())) if float(t.get_text()) < 100000 else 'n' for t in ax12.get_xticklabels()])
fig12.savefig(f"{figures_path}/uav_combined_arc.pdf", bbox_inches='tight')

In [ ]:
fig11, ax11 = plt.subplots(figsize=(10, 4))
sns.pointplot(x='Number of Rounds', y='Planning Time (s)', hue="Problem Name", data=uav_data, ax=ax11, hue_order = uav_table_order,linewidth = 1.5, errorbar=None)
sns.move_legend(ax11, "lower center", bbox_to_anchor=(0.45, 1), ncol=3, title=None, frameon=False)
ax11.set_xticklabels(['{:.0f}'.format(float(t.get_text())) if float(t.get_text()) < 100000 else 'n' for t in ax11.get_xticklabels()])
fig11.savefig(f"{figures_path}/uav_combined_time.pdf", bbox_inches='tight')

In [ ]:
result = []
for file_name in os.listdir(data_path+"/road_data"):
    components = file_name.split(".")[0].split("_")
    if len(components) > 3 and components[0] == 'sensing' and components[3] == '50':
        suffix = "-2" if len(components) > 4 else ""
        name = f"road-{components[2]}-50{suffix}"
        mat = np.load(f"{data_path}/road_data/{file_name}")
        result.append((name, mat.sum(axis=1).mean(), mat.sum(axis=0).mean()))
        

In [ ]:
sensing_df = pd.DataFrame(result, columns=["Instance Name", "Average Number of Scenarios per Node", "Average Nodes Per Scenario"])

In [ ]:
sensing_df.set_index("Instance Name", inplace=True)
sensing_df = sensing_df.reindex(hue_order)

In [ ]:
sensing_df

In [ ]:
s = sensing_df.style.format(precision=2)
buf=f'{tables_path}/sensing_matrix_summary.tex'
print(s.to_latex(
    hrules=True, 
    multicol_align="c", 
    environment='table', 
    caption='Statistics of Sensing Matrix',
    label='tab:sensing_stats', 
    position='htbp', 
    position_float='centering'))

In [ ]:
sns.pointplot(x='Number of Rounds', y='Iteration Taken', hue="Problem Name", data=uav_data, hue_order = uav_table_order,linewidth = 1.5, errorbar=None)
plt.show()

In [ ]:
road_data["Iteration Taken"]

In [ ]:
road_data.groupby("Number of Rounds")['Planning Time (s)'].mean()

In [ ]:
road_data.groupby("Number of Rounds")['Relative Cost Per Scenario (vs. Fully Adaptive)'].mean()

In [ ]:
122.319875/7.625560

In [ ]:
(12.906671-  0.639472)/ 0.639472

In [ ]:
6*16

INFORMS PAPER PLOTS


In [ ]:
fig_ir, axes_ir = plt.subplots(nrows=1,ncols=2, figsize=(20, 6))
axr1 = axes_ir[0]
axr2 = axes_ir[1]
sns.pointplot(x='Number of Rounds', y='Planning Time (s)', hue="Problem Name", data=road_data, legend="full", ax=axr1, hue_order = hue_order,linewidth = 1.5, errorbar=None)
axr1.set_xticklabels(['{:.0f}'.format(float(t.get_text()))  if float(t.get_text()) < 100000 else 'n' for t in axr1.get_xticklabels()])
axr1.set_ylabel("Average Planning Time (s)")

hue_order = sorted(road_data['Problem Name'].unique())
sns.pointplot(x='Number of Rounds', y='Relative Cost Per Scenario (vs. Fully Adaptive)', hue="Problem Name", data=road_data, legend=False, ax=axr2, hue_order = hue_order,linewidth = 1.5, errorbar=None)
axr2.set_xticklabels(['{:.0f}'.format(float(t.get_text())) if float(t.get_text()) < 100000 else 'n' for t in axr2.get_xticklabels()])
axr2.set_ylabel("Average Relative Cost Per Scenario")
# handles, labels = axr1.get_legend_handles_labels()
# fig_ir.legend(handles, labels, loc='upper center', ncols=5, frameon=False, bbox_to_anchor=(0.5, 1.1))
sns.move_legend(axr1, "upper center", bbox_to_anchor=(1, 1.25), ncol=5, title=None, frameon=False)

fig_ir.savefig(f"{figures_path}/road_informs.pdf", bbox_inches='tight')

In [ ]:
# fig4, ax4 = plt.subplots(figsize=(10, 6))
# sns.pointplot(x='Number of Rounds', y='Path Cost', hue="Problem Name", data=uav_data, legend="full", ax=ax4, hue_order = uav_table_order,linewidth = 1.5, errorbar=None)
# sns.move_legend(ax4, "lower center", bbox_to_anchor=(0.47, 1), ncol=3, title=None, frameon=False)
# ax4.set_xticklabels(['{:.0f}'.format(float(t.get_text()))  if float(t.get_text()) < 100000 else 'n' for t in ax4.get_xticklabels()])
# fig4.savefig(f"{figures_path}/uav_cost.pdf", bbox_inches='tight')

fig_iu, axes_iu = plt.subplots(nrows=1,ncols=2, figsize=(20, 6))
axu1 = axes_iu[0]
axu2 = axes_iu[1]
sns.pointplot(x='Number of Rounds', y='Planning Time (s)', hue="Problem Name", data=uav_data, legend="full", ax=axu1, hue_order = uav_table_order,linewidth = 1.5, errorbar=None)
axu1.set_xticklabels(['{:.0f}'.format(float(t.get_text()))  if float(t.get_text()) < 100000 else 'n' for t in axu1.get_xticklabels()])
axu1.set_ylabel("Average Planning Time (s)")

sns.pointplot(x='Number of Rounds', y='Relative Cost Per Scenario (vs. Fully Adaptive)', hue="Problem Name", data=uav_data, legend=False, ax=axu2, hue_order = uav_table_order,linewidth = 1.5, errorbar=None)
axu2.set_xticklabels(['{:.0f}'.format(float(t.get_text())) if float(t.get_text()) < 100000 else 'n' for t in axu2.get_xticklabels()])
axu2.set_ylabel("Average Relative Cost Per Scenario")
# handles, labels = axu1.get_legend_handles_labels()
# fig_iu.legend(handles, labels, loc='upper center', ncols=6, frameon=False, bbox_to_anchor=(0.5, 1))
sns.move_legend(axu1, "upper center", bbox_to_anchor=(1.1, 1.15), ncol=6, title=None, frameon=False)

fig_iu.savefig(f"{figures_path}/uav_informs.pdf", bbox_inches='tight')